In [2]:
# Do more highly-recommended stocks outperform lowly-rated ones?

import pandas as pd
import numpy as np
import yfinance as yf
import warnings
warnings.filterwarnings("ignore")
import datetime
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cm
import sqlite3
from IPython.display import display
from sklearn.metrics import r2_score
import math
pd.set_option('display.max_columns',500)

In [42]:
# Get a list of all SP500 companies
query=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
sp500 = list(query[0]['Symbol'].values)

In [16]:
def get_history_and_recommendation_data(stock_ticker,period='5y',days_out=180,min_recommendations=5):
    # Prepare yfinance
    print('Results for: '+stock_ticker)
    stock=yf.Ticker(stock_ticker)
    
    # Get historical trading data
    try:
        history=stock.history(period=period)
    except:
        print('No history data for '+stock_ticker)
        return
    
    # Get recommendations
    try:
        recommendations=stock.recommendations.reset_index().sort_values(by=['Firm','Date'])
    except:
        print('No analyst recommendation data for '+stock_ticker)
        return
    recommendations['Next Recommendation']=recommendations.groupby('Firm')['Date'].shift(-1)
    recommendations[str(days_out)+' Days From']=recommendations['Date']+pd.Timedelta(days=days_out)
    recommendations['Good Until']=recommendations\
        [['Next Recommendation',str(days_out)+' Days From']]\
            .min(axis=1) # Define how long a recommendation lasts for - lowest of [days_out] or
                # [Until a new recommendation by that firm]

    # Process the recommendation values
    print('Recommendation values: '\
          +str(sorted(recommendations['To Grade'].dropna().unique()))) # Print out the unique recommendation grades
    assign_values = {'Buy':4, 'Equal-Weight':3, 'Hold':3, 'Market Perform':3, 'Neutral':3,
       'Outperform':4, 'Overweight':4, 'Peer Perform':3, 'Positive':4, 'Reduce':2,
       'Sector Perform':3, 'Sector Weight':3, 'Sell':1, 'Strong Buy':5, 'Strong Sell':1, 'Underperform':1,
       'Underweight':1} # Create numeric representations of the recommendations
    recommendations['Assigned Value']=\
    recommendations['To Grade'].replace(assign_values) # Give the numeric representations a column
    
    # Store the dataframes in sqlite memory
    conn = sqlite3.connect(':memory:')
    history.to_sql('history', conn, index=True)
    recommendations.to_sql('recommendations', conn, index=False)
    
    # Get average analyst assigned scores by day using SQL
    qry = '''
    select h.open, h.high, h.low, h.close, h.volume, date(h.date) as Date, avg(r.'assigned value') as 'Average Assigned Value',
    count(r.'assigned value') as 'Count of Ratings'
    from history h
    left join ( 
    select * from recommendations r 
    where r.firm in 
    (select firm from recommendations group by firm having count(date)>='''+str(int(min_recommendations))+''')
        -- Join recommendations, but only for firms with at least minimum recommendations recommendations
    ) r
    on date(h.date) between date(r.date) and date(r.'good until')
    group by h.open, h.high, h.low, h.close, h.volume, date(h.date)
        '''
    df = pd.read_sql_query(qry, conn).sort_values(by='Date')
    df['Date']=pd.to_datetime(df['Date'])
    return df

In [18]:
aapl=get_history_and_recommendation_data('AAPL')
aapl

Results for: AAPL
Recommendation values: ['', 'Buy', 'Equal-Weight', 'Equal-weight', 'Hold', 'Long-Term Buy', 'Long-term Buy', 'Market Outperform', 'Market Perform', 'Negative', 'Neutral', 'Outperform', 'Overweight', 'Peer Perform', 'Perform', 'Positive', 'Reduce', 'Sector Outperform', 'Sector Perform', 'Sector Weight', 'Sell', 'Strong Buy', 'Underperform', 'Underweight']


,Open,High,Low,Close,Volume,Date,Average Assigned Value,Count of Ratings
0,31.412197,31.586868,31.334303,31.461763,92141600,2017-02-13,3.844444,45
1,31.504256,31.886640,31.452327,31.870119,132904800,2017-02-14,3.844444,45
3,31.988130,32.165160,31.775692,31.985767,142492400,2017-02-15,3.844444,45
4,32.023538,32.077827,31.827625,31.948008,90338400,2017-02-16,3.844444,45
2,31.889007,32.061315,31.889007,32.035351,88792800,2017-02-17,3.844444,45
...,...,...,...,...,...,...,...,...
1237,172.860001,173.949997,170.949997,171.660004,77251200,2022-02-07,3.857143,21
1230,171.729996,175.350006,171.429993,174.830002,74829200,2022-02-08,3.857143,21
1249,176.050003,176.649994,174.899994,176.279999,71285000,2022-02-09,3.857143,21
1239,174.139999,175.479996,171.550003,172.119995,90865900,2022-02-10,3.857143,21


In [52]:
def get_stock_rating_and_return(df,minimum_average_ratings=3,trim_start=100):
    df=df.iloc[100:,:]
    if df['Count of Ratings'].mean() < minimum_average_ratings:
        print('Not enough ratings for this stock.')
        return None, None
    else:
        first_open=df.iloc[[0]]['Open'].values[0]
        last_close=df.iloc[[-1]]['Close'].values[0]
        total_return_fraction=last_close/first_open
        average_rating=df['Average Assigned Value'].mean()
        count_days=df.shape[0]
        return_rate = 100*\
            (total_return_fraction**(1/count_days)-1) # Get the daily 
                            # return rate for the period by taking the n-th root of n-day
                                # cumulative stock movements and subtracting 1 (remember the compound interest equation)
    return return_rate, average_rating

In [53]:
import random
stocks = random.sample(sp500,50)
store_values = []
for stock in stocks:
    returns,rating = get_stock_rating_and_return(get_history_and_recommendation_data(stock))
    store_values.append([stock,returns,rating])

Results for: TFC
Recommendation values: ['Buy', 'Equal-Weight', 'Market Perform', 'Neutral', 'Outperform', 'Overweight', 'Underperform']
Not enough ratings for this stock.


In [47]:
store_values

[['NFLX', 0.08257080148466045, 3.5644940360508692],
 ['FDX', 0.01032434118108938, 3.668589152490912],
 ['RCL', -0.016954628141763184, 3.464143666199191],
 ['BXP', 0.011169003762057095, 3.0786165301480226],
 ['MPWR', 0.12942247349450042, 3.9360775572811812]]

In [11]:
def process_individual_stock(stock_ticker,period='5y',days_out=180,color_method='Auto',\
                             qlow=0.15,qhigh=0.85,vmin=2.75,vmax=3.75,\
                             returns_method='Period',\
                             min_returns=5,split_size=50):
    # Prepare yfinance
    print('Results for: '+stock_ticker)
    stock=yf.Ticker(stock_ticker)
    
    # Get historical trading data
    try:
        history=stock.history(period=period)
    except:
        print('No history data for '+stock_ticker)
        return
    
    # Get recommendations
    days_out = days_out
    try:
        recommendations=stock.recommendations.reset_index().sort_values(by=['Firm','Date'])
    except:
        print('No analyst recommendation data for '+stock_ticker)
        return
    recommendations['Next Recommendation']=recommendations.groupby('Firm')['Date'].shift(-1)
    recommendations[str(days_out)+' Days From']=recommendations['Date']+pd.Timedelta(days=days_out)
    recommendations['Good Until']=recommendations\
        [['Next Recommendation',str(days_out)+' Days From']]\
            .max(axis=1) # Define how long a recommendation lasts for - lowest of [days_out] or
                # [Until a new recommendation by that firm]

    # Process the recommendation values
    print('Recommendation values: '\
          +str(sorted(recommendations['To Grade'].dropna().unique()))) # Print out the unique recommendation grades
    assign_values = {'Buy':4, 'Equal-Weight':3, 'Hold':3, 'Market Perform':3, 'Neutral':3,
       'Outperform':4, 'Overweight':4, 'Peer Perform':3, 'Positive':4, 'Reduce':2,
       'Sector Perform':3, 'Sector Weight':3, 'Sell':1, 'Strong Buy':5, 'Underperform':1,
       'Underweight':1} # Create numeric representations of the recommendations
    recommendations['Assigned Value']=\
    recommendations['To Grade'].replace(assign_values) # Give the numeric representations a column
    
    # Store the dataframes in sqlite memory
    conn = sqlite3.connect(':memory:')
    history.to_sql('history', conn, index=True)
    recommendations.to_sql('recommendations', conn, index=False)
    
    # Get average analyst assigned scores by day using SQL
    qry = '''
    select h.open, h.high, h.low, h.close, h.volume, date(h.date) as Date, avg(r.'assigned value') as 'Average Assigned Value'
    from history h
    left join ( 
    select * from recommendations r 
    where r.firm in 
    (select firm from recommendations group by firm having count(date)>=5)
        -- Join recommendations, but only for firms with at least 5 recommendations
    ) r
    on date(h.date) between date(r.date) and date(r.'good until')
    group by h.open, h.high, h.low, h.close, h.volume, date(h.date)
        '''
    df = pd.read_sql_query(qry, conn).sort_values(by='Date')
    df['Date']=pd.to_datetime(df['Date'])

        
    # Make a plot with background color estimating recommendation sentiment for the periods
    # This considers the average recommendation overall
    fig,ax = plt.subplots()
    df.plot(figsize=(16,16),x='Date',y='Close',ax=ax)
    # # Originally used a list color map but change to a scalar map
    colormap=colors.LinearSegmentedColormap.from_list('custom',['Red','Yellow','Green'])
    # vmin, vmax = 2.5, 4.5 # Manual color definition
    if color_method == 'Auto':
        qlow = qlow
        qhigh = qhigh
        vmin = df['Average Assigned Value'].quantile(qlow) # Color definition using quantiles (low)
        vmax = df['Average Assigned Value'].quantile(qhigh) # Color definition using quantiles (high)
    elif color_method == 'Manual':
        vmin = vmin
        vmax = vmax
    else:
        print('Error: Please choose Auto or Manual to define your color scheme.')
        return
    normalize = colors.Normalize(vmin=vmin,vmax=vmax)
    scalarmappaple = cm.ScalarMappable(norm=normalize, cmap=colormap)
    for idx, row in df.iterrows():
        ax.axvspan(row['Date'],row['Date']+pd.DateOffset(days=1),\
                   alpha=0.2,color=scalarmappaple.to_rgba(row['Average Assigned Value']))
    cbar=plt.colorbar(scalarmappaple,label='Average Analyst Recommendation',extend='both',fraction=0.046, pad=0.04)
    plt.show()
    
    # Get a correlation matrix of the current columns
    df['Returns']=(df['Close']-df['Close'].shift(1))/df['Close'].shift(1) # Day returns
    print('Correlation matrix for all data:')
    display(df.corr())
    
    if returns_method == 'Recommendation Moves':
        # Process the dataframe to get columns that can be grouped on the
                # analyst average recommendation to show how it affects returns
        df['Track Difference Up'] = np.where(df['Average Assigned Value']==df['Average Assigned Value'].shift(1),0,1)
        df['Track Difference Down'] = np.where(df['Average Assigned Value']==df['Average Assigned Value'].shift(-1),0,1)
        df['Track Difference Unique'] = df['Track Difference Up'].cumsum()
        df['First Open'] = np.where(df['Track Difference Up']==1,df['Open'],None)
        df['Last Close'] = np.where(df['Track Difference Down']==1,df['Close'],None)

        # Get a grouped dataframe with returns data by average analyst recommendation
        returns_df = df.groupby(['Track Difference Unique','Average Assigned Value'])\
            .agg({'First Open':'min','Last Close':'max','Returns':'count'}).reset_index()
        returns_df['Total Return Fraction']=returns_df['Last Close']/returns_df['First Open']
        returns_df['Daily Return']=100*\
            ((returns_df['Total Return Fraction']**(1/returns_df['Returns']))-1) # Get the daily 
                            # return rate for the period by taking the n-th root of n-day
                                # cumulative stock movements and subtracting 1 (remember the compound interest equation)

        # Create a dataframe that has at least the minimum days
                    # (otherwise 1-day moves could yield high variance, potentially leading to false conclusions)
                    # Plot and correlate returns and ratings of that dataframe
        min_returns = min_returns
        min_returns_df = returns_df[returns_df['Returns']>=min_returns]
        min_returns_df.plot(x='Average Assigned Value',y='Daily Return',kind='scatter',figsize=(12,12))
        m,b = np.polyfit(x=min_returns_df['Average Assigned Value'],y=min_returns_df['Daily Return'], deg=1)
        plt.plot(min_returns_df['Average Assigned Value'],min_returns_df['Average Assigned Value']*m+b)
        title = 'R2 Score: '+str(round(r2_score(min_returns_df['Daily Return'],min_returns_df['Average Assigned Value']*m+b),4))
        plt.title(title)
        plt.show()
        print('Correlation matrix for processed returns data:')
        display(min_returns_df.corr())
#         display(min_returns_df)
    
    elif returns_method == 'Period':
        # Split the stock into periods and find the return rate during that period as well as the average rating
        split_size = split_size
        split_bins = math.ceil(df.shape[0]/split_size)
        df = pd.read_sql_query(qry, conn).sort_values(by='Date')
        df['Date']=pd.to_datetime(df['Date'])
        df['Period Group'] = pd.cut(df['Date'],split_bins)
        df['First Open'] = df.groupby('Period Group').head(1)['Open']
        df['Last Close'] = df.groupby('Period Group').tail(1)['Close']
#         display(df)
        
        # Get a grouped dataframe with returns data for that period to compare against average analyst recommendation
        returns_df = df.groupby(['Period Group'])\
        .agg({'First Open':'min','Last Close':'max','Date':'count','Average Assigned Value':'mean'}).reset_index()
        returns_df['Total Return Fraction']=returns_df['Last Close']/returns_df['First Open']
        returns_df['Daily Return']=100*\
            ((returns_df['Total Return Fraction']**(1/returns_df['Date']))-100)-100 # Get the daily 
                            # return rate for the period by taking the n-th root of n-day
                                # cumulative stock movements and subtracting 1 (remember the compound interest equation)

        # Create a dataframe that has at least the minimum days. Get the return for each period bin.
                    # Then plot and correlate that dataframe
        min_returns_df = returns_df[returns_df['Date']\
                                    >=min(split_size-1,split_size*0.9)] # Each bin
                            # for plotting and correlation needs to be at least 90% the size of the split_size
        min_returns_df.plot(x='Average Assigned Value',y='Daily Return',kind='scatter',figsize=(12,12))
        m,b = np.polyfit(x=min_returns_df['Average Assigned Value'],y=min_returns_df['Daily Return'], deg=1)
        plt.plot(min_returns_df['Average Assigned Value'],min_returns_df['Average Assigned Value']*m+b)
        title = 'R2 Score: '+\
            str(round(r2_score(min_returns_df['Daily Return'],min_returns_df['Average Assigned Value']*m+b),4))
        plt.title(title)
        plt.show()
        print('Correlation matrix for processed returns data:')
        display(min_returns_df.corr())
#         display(min_returns_df)
    
    else:
        print('Error: Please choose Period or Recommendation Moves as your method of returns analysis.')
        return
    
    print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')